In [1]:
import csv
import math
import matplotlib.pyplot as plt
import numpy
import random
import sklearn
import string
from collections import defaultdict
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [2]:
# convert csv to list of dicts
with open('final_df.csv') as f:
    data = [{k: v for k, v in row.items()} for row in csv.DictReader(f, skipinitialspace=True)]

In [3]:
data[0]

{'uid_x': '255938',
 'profile': 'DesolatePsyche',
 'anime_uid': '34096',
 'text': 'First things first My reviews system is explained on a blog entry Which can be found through my profile   Im going to keep this review more of a opinion of Gintamas overall and then this season specific  Anyhow What I have always loved regarding Gintama is its content of everything I love the comedy its absurd random can be vile dirty sweet anyhow everything Have laughed countless times in this franchise Also the humor they have also is heavily reference based aka parodies of different anime shows manga live stuff real world anime production and so on Anyhow comedyparody side of this franchise i absolutely love  Now nd side of this show is the serious dramas epic battle shounens and so on There are arcs that are fully comedy arcs that are fully serious and mixtures of both Serious side is usually quite dramatic and managed to somewhat tear me up now and then Whilst the action sequences are absolute bliss

In [8]:
punctuation = set(string.punctuation)
stopwordSet = set(stopwords.words('english'))

In [9]:
def text_process(reviewText):
    nopunc = [i for i in reviewText if i not in punctuation]
    nopunc_text = ''.join(nopunc)
    return [i.lower() for i in nopunc_text.split() if i.lower() not in stopwordSet]

In [10]:
def accuracy(predictions, labels):
    correct = 0
    for i in range(len(predictions)):
        if predictions[i] == labels[i]:
            correct += 1
    return correct/len(labels)

In [11]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [23]:
X = [' '.join(text_process(d['text'])) for d in data]
y = [d['score_x'] for d in data]
y = list(map(int, y))

In [57]:
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1,5), analyzer='char')

In [58]:
X_tfidf = tfidf.fit_transform(X)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=0)

In [60]:
from sklearn.svm import LinearSVC
clf = LinearSVC(class_weight='balanced', verbose=2)
clf.fit(X_train, y_train)

[LibLinear].................*.*
optimization finished, #iter = 182
Objective value = -15615.704596
nSV = 24739
................*
optimization finished, #iter = 168
Objective value = -19830.796686
nSV = 31361
..........*
optimization finished, #iter = 101
Objective value = -23407.778808
nSV = 41156
.........*
optimization finished, #iter = 99
Objective value = -25642.175451
nSV = 48929
.......*
optimization finished, #iter = 73
Objective value = -27416.127611
nSV = 58854
.....*
optimization finished, #iter = 57
Objective value = -29019.272049
nSV = 69129
.....*
optimization finished, #iter = 53
Objective value = -29950.331288
nSV = 78525
.....*
optimization finished, #iter = 52
Objective value = -30301.958480
nSV = 81917
.....*
optimization finished, #iter = 51
Objective value = -29625.299146
nSV = 77833
.....*
optimization finished, #iter = 51


LinearSVC(class_weight='balanced', verbose=2)

Objective value = -23958.313037
nSV = 57935


In [61]:
y_pred = clf.predict(X_test)

In [62]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.20      0.44      0.27       907
           2       0.11      0.27      0.16       944
           3       0.17      0.26      0.20      1744
           4       0.13      0.19      0.16      1792
           5       0.18      0.19      0.19      2599
           6       0.23      0.19      0.21      3706
           7       0.32      0.24      0.27      5731
           8       0.36      0.24      0.29      7047
           9       0.36      0.24      0.29      7343
          10       0.51      0.69      0.58      7343

    accuracy                           0.32     39156
   macro avg       0.26      0.29      0.26     39156
weighted avg       0.33      0.32      0.31     39156



In [63]:
MSE(y_pred, y_test)

4.679691490448462